In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-x3kntfer
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-x3kntfer
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369393 sha256=38cbea737ce025b98e7ee22bdfef6733f5fa5fc0fe42c762fe01af09ec53ac5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ltcrjasj/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


In [2]:
import numpy as np
import torch
from pkg_resources import packaging
import clip
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from collections import OrderedDict
import torch
import skimage
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import torchvision
from torchvision.datasets import EuroSAT
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import random

print("Torch version:", torch.__version__)
model, preprocess = clip.load("RN50") #The model variable will hold the loaded model, while the preprocess variable will store the preprocessing function for preparing the input data.
model.cuda().eval()
input_resolution = model.visual.input_resolution # (size) of the input images expected by the model.
context_length = model.context_length # the length of the tokenized text input that the model can process.
vocab_size = model.vocab_size # the size of the vocabulary used by the model.

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Torch version: 2.0.1+cu117
Model parameters: 102,007,137
Input resolution: 224
Context length: 77
Vocab size: 49408


Main code

In [4]:
pip install --upgrade torch


Requirement already up-to-date: torch in ./.local/lib/python3.8/site-packages (2.0.1)
Note: you may need to restart the kernel to use updated packages.


In [3]:
models = ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
models = ['ViT-B/32']
for k in range(len(models)):
    model, preprocess = clip.load(models[k])
    model.cuda().eval()
    Eurosat = EuroSAT(root='~/.cache', transform=preprocess, download=True)
    classes = Eurosat.classes
    classes[0] = 'annual crop land'
    classes[2] = 'brushland or shrubland'
    classes[4] = 'highway or road'
    classes[5] = 'pasture land'
    classes[6] = 'permanant crop land'
    dataloader = DataLoader(Eurosat, batch_size=32, shuffle=True)
    device = "cuda"
    count = 0
    top_k_accuracy = 1
    count_total = 0
    for i, (images, labels) in enumerate(dataloader):   
        label = []
        # Iterate over the dataset and store images and labels
        for j in range(len(labels)):
        # Retrieve image and label at index i
            label.append(classes[labels[j]])
        with torch.no_grad():
            image_input = torch.tensor(np.stack(images)).cuda()
            text_descriptions = [f"a centered satellite photo of a {labe}" for labe in classes]
            text_tokens = clip.tokenize(text_descriptions).cuda()
            image_features = model.encode_image(image_input).float() # image encoder
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features = model.encode_text(text_tokens).float() # text encoder
            text_features /= text_features.norm(dim=-1, keepdim=True)
        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)   
        for i in range(len(label)):
            count_total += 1
            values, indices = text_probs[i].topk(top_k_accuracy)
            for value, index in zip(values, indices):
                #print (Eurosat.classes[index])
                if Eurosat.classes[index] == label[i]:
                    count += 1
                    break
    print(models[k])
    print ("Accuracy = ", count/len(Eurosat))
    print("\n")


ViT-B/32
Accuracy =  0.3382222222222222




In [13]:
image_features[0]
text_features[0]

tensor([-1.1024e-02,  1.1208e-02,  3.1070e-02,  3.0421e-02, -9.6726e-03,
        -2.2579e-02, -4.9175e-02, -1.8430e-02,  4.8614e-02,  9.2222e-03,
         3.3699e-02,  2.9885e-03,  9.3034e-03, -4.2382e-03, -2.2801e-02,
        -1.7809e-02,  3.2399e-02,  1.0618e-02,  1.4959e-02,  1.0293e-02,
        -2.1161e-02, -2.5222e-02,  5.3192e-02, -1.0566e-02, -2.5134e-02,
         7.8820e-04, -3.3817e-02,  2.5222e-02,  3.0066e-02, -3.9399e-02,
         5.6153e-03, -3.8690e-02,  3.3492e-02, -5.8515e-03, -5.4964e-02,
         4.0610e-03,  1.1356e-02, -1.4605e-02, -1.8075e-02, -6.2170e-03,
         1.7174e-02, -1.2870e-02,  5.1095e-02,  7.2419e-02,  1.9877e-02,
        -2.0674e-03, -1.8887e-02,  2.5119e-02,  1.9581e-02,  1.4915e-02,
        -3.4869e-03,  2.2756e-02,  9.8276e-03, -2.3738e-03,  1.9700e-02,
         5.5466e-02, -1.3460e-02,  3.3020e-02, -5.0504e-03, -1.2390e-02,
        -1.6111e-02,  7.6051e-04,  2.5326e-02,  2.1058e-02,  1.9463e-02,
         7.5165e-03, -2.1309e-02, -4.7255e-03, -1.5

In [10]:
Eurosat = EuroSAT(root='~/.cache', transform=preprocess, download=True)

In [14]:
Eurosat[20000]
classes

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

Steps by steps

Construct images dataset, images list contain processed images, labels contains corresponding labels


In [14]:

Imageset = ImageFolder(root="~/.cache", transform=preprocess, download=True)

TypeError: __init__() got an unexpected keyword argument 'download'

In [33]:

Eurosat = EuroSAT(root='~/.cache', transform=preprocess, download=True)
classes = Eurosat.classes
classes

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [34]:
len(Eurosat)

27000

In [11]:

random_seed = 42
random.seed(random_seed)
sample_number = 10000

# Randomly choose 1000 indices from the dataset
indices = random.sample(range(len(Eurosat)), sample_number)

# Create a subset using the randomly chosen indices
random_subset = Subset(Eurosat, indices)

In [12]:
eurosat = random_subset
eurosat[0]

(tensor([[[-0.7850, -0.7850, -0.7850,  ..., -0.8288, -0.8288, -0.8288],
          [-0.7850, -0.7850, -0.7850,  ..., -0.8288, -0.8288, -0.8288],
          [-0.7850, -0.7850, -0.7850,  ..., -0.8288, -0.8288, -0.8288],
          ...,
          [-0.6390, -0.6244, -0.6244,  ..., -0.5806, -0.5806, -0.5806],
          [-0.6536, -0.6390, -0.6390,  ..., -0.5806, -0.5806, -0.5806],
          [-0.6536, -0.6390, -0.6390,  ..., -0.5806, -0.5806, -0.5806]],
 
         [[-0.3564, -0.3564, -0.3564,  ..., -0.4614, -0.4614, -0.4614],
          [-0.3564, -0.3564, -0.3564,  ..., -0.4614, -0.4614, -0.4614],
          [-0.3564, -0.3564, -0.3564,  ..., -0.4614, -0.4614, -0.4614],
          ...,
          [-0.2363, -0.2363, -0.2513,  ..., -0.4614, -0.4614, -0.4614],
          [-0.2513, -0.2513, -0.2663,  ..., -0.4614, -0.4614, -0.4614],
          [-0.2513, -0.2513, -0.2663,  ..., -0.4614, -0.4614, -0.4614]],
 
         [[-0.1009, -0.1009, -0.1009,  ..., -0.1009, -0.1009, -0.1009],
          [-0.1009, -0.1009,

In [7]:
images = []
labels = []

# Iterate over the dataset and store images and labels
for i in range(len(eurosat)):
    image, label = eurosat[i]  # Retrieve image and label at index i
    images.append(image)
    labels.append(label)

In [8]:

for i in range(len(labels)):
    labels[i] = classes[labels[i]]
labels

['Residential',
 'Forest',
 'AnnualCrop',
 'SeaLake',
 'Highway',
 'HerbaceousVegetation',
 'HerbaceousVegetation',
 'Forest',
 'SeaLake',
 'Forest',
 'River',
 'SeaLake',
 'PermanentCrop',
 'AnnualCrop',
 'Residential',
 'Industrial',
 'AnnualCrop',
 'AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'HerbaceousVegetation',
 'PermanentCrop',
 'Residential',
 'AnnualCrop',
 'PermanentCrop',
 'HerbaceousVegetation',
 'River',
 'Residential',
 'River',
 'PermanentCrop',
 'Industrial',
 'HerbaceousVegetation',
 'Pasture',
 'Residential',
 'Highway',
 'SeaLake',
 'AnnualCrop',
 'SeaLake',
 'SeaLake',
 'Forest',
 'River',
 'Industrial',
 'Highway',
 'Highway',
 'Forest',
 'HerbaceousVegetation',
 'SeaLake',
 'Highway',
 'Forest',
 'Forest',
 'Industrial',
 'Forest',
 'Industrial',
 'Highway',
 'Residential',
 'HerbaceousVegetation',
 'SeaLake',
 'AnnualCrop',
 'River',
 'Pasture',
 'PermanentCrop',
 'Forest',
 'Industrial',
 'AnnualCrop',
 'PermanentCrop',
 'Highway',
 'Residential',
 'Resi

In [14]:
images

tensor([[[[-0.6244, -0.6244, -0.6244,  ..., -1.0331, -1.0331, -1.0331],
          [-0.6244, -0.6244, -0.6244,  ..., -1.0331, -1.0331, -1.0331],
          [-0.6244, -0.6244, -0.6244,  ..., -1.0331, -1.0331, -1.0331],
          ...,
          [-0.3324, -0.3324, -0.3324,  ..., -0.5076, -0.5076, -0.5076],
          [-0.3324, -0.3324, -0.3324,  ..., -0.4930, -0.4930, -0.4930],
          [-0.3324, -0.3324, -0.3324,  ..., -0.4784, -0.4784, -0.4784]],

         [[-0.3714, -0.3714, -0.3714,  ..., -0.4914, -0.4914, -0.4914],
          [-0.3714, -0.3714, -0.3714,  ..., -0.4914, -0.4914, -0.4914],
          [-0.3714, -0.3714, -0.3714,  ..., -0.4914, -0.4914, -0.4914],
          ...,
          [-0.2663, -0.2663, -0.2663,  ..., -0.3264, -0.3414, -0.3414],
          [-0.2663, -0.2663, -0.2663,  ..., -0.3264, -0.3264, -0.3264],
          [-0.2663, -0.2663, -0.2663,  ..., -0.3264, -0.3264, -0.3114]],

         [[-0.1720, -0.1720, -0.1720,  ..., -0.3284, -0.3284, -0.3284],
          [-0.1720, -0.1720, -

Transform froms of image

In [9]:
with torch.no_grad():
    image_input = torch.tensor(np.stack(images)).cuda()

construct text dataset

In [10]:
with torch.no_grad():
    text_descriptions = [f"This is a photo of a {label}" for label in Eurosat.classes]
    text_tokens = clip.tokenize(text_descriptions).cuda()

Pass two datasets to encoders (feature representation)

In [11]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float() # image encoder
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features = model.encode_text(text_tokens).float() # text encoder
    text_features /= text_features.norm(dim=-1, keepdim=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 718.00 MiB (GPU 0; 9.78 GiB total capacity; 8.75 GiB already allocated; 425.81 MiB free; 8.77 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Calculate similarities

In [12]:
text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

In [13]:
count = 0
for i in range(sample_number):
    values, indices = text_probs[i].topk(1)
    for value, index in zip(values, indices):
        if Eurosat.classes[index] == labels[i]:
            count += 1
       # print(f"{Eurosat.classes[index]:>16s}: {100 * value.item():.2f}%")
    #print("True label: ",labels[ind])
print ("Accuracy: ", 100*count/sample_number, "%")

Accuracy:  34.93333333333333 %


In [30]:
Eurosat = EuroSAT(root='~/.cache', transform=preprocess, download=True)
classes = Eurosat.classes
classes
dataloader = DataLoader(Eurosat, batch_size=32, shuffle=True)
device = "cuda"
count = 0
for i, (images, labels) in enumerate(dataloader):
    
    # Move images and labels to the GPU
    label = []

    # Iterate over the dataset and store images and labels
    for j in range(len(labels)):
    # Retrieve image and label at index i
        label.append(classes[labels[j]])
    with torch.no_grad():
        image_input = torch.tensor(np.stack(images)).cuda()
        text_descriptions = [f"This is a photo of a {labe}" for labe in classes]
        text_tokens = clip.tokenize(text_descriptions).cuda()
        image_features = model.encode_image(image_input).float() # image encoder
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features = model.encode_text(text_tokens).float() # text encoder
        text_features /= text_features.norm(dim=-1, keepdim=True)
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)   
    for i in range(len(label)):
        values, indices = text_probs[i].topk(1)
        for value, index in zip(values, indices):
            #print (Eurosat.classes[index])
            if Eurosat.classes[index] == label[i]:
                count += 1
    

    # Encode the images using the CLIP model


In [35]:
count/27000

0.3411111111111111

In [ ]:
# pesudo-code, cant run

images = [] # put images of EuroSAT into the list
Label = []	# put labels of images into the list
preprocess(images)
image_input = torch.tensor(np.stack(images)).cuda() # Transform images to tensor format

# pad the label to be text
text_descriptions = [f"This is a photo of a {label}" for label in EuroSAT.classes]

# converts the text into a sequence of numerical
# tokens that can be understood by the CLIP model.
text_tokens = clip.tokenize(text_descriptions).cuda()

with torch.no_grad():
    image_features = model.encode_image(image_input).float() # image encoder
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features = model.encode_text(text_tokens).float() # text encoder
    text_features /= text_features.norm(dim=-1, keepdim=True)

# similarities from every image to every text
text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# This means that for the 3rd image,
# the five most likely counterparts of the text
values, indices = text_probs[2].topk(5)


for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

# iterate through text_probs to calculate accuracy
calculate top-5/top-1 accuracy